# Westmeyer Lab Napari Implementation for EMcapsulins

## Imports

In [1]:
import napari
from magicgui import magic_factory
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops, profile_line
from skimage import exposure, measure, filters
from skimage.draw import disk
from scipy.signal import find_peaks
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
#from plotly.io import to_html
#viewer = napari.view_image(nuclei, colormap='magma')

## Functions

### Extract and filter blobs

In [2]:
# Global variables to store the stacked regions and number of images to select
stacked_regions = None
n_select = 0

@magic_factory(image_layer={'label': 'Select Image Layer'},
               mask_layer={'label': 'Select Mask Layer'})
def test_blob_centers(image_layer: 'napari.layers.Image', mask_layer: 'napari.layers.Image', viewer: 'napari.Viewer') -> None:
    global stacked_regions

    img_data = image_layer.data
    mask_data = mask_layer.data

    labeled_mask = label(mask_data)

    regions = []
    for region in regionprops(labeled_mask):
        centroid_y, centroid_x = map(int, region.centroid)
        min_y, max_y = centroid_y - 25, centroid_y + 25
        min_x, max_x = centroid_x - 25, centroid_x + 25

        region_img = img_data[max(min_y, 0):max_y, max(min_x, 0):max_x]

        if region_img.shape == (50, 50):
            regions.append(region_img)

    if regions:
        stacked_regions = np.array(regions)
        viewer.add_image(stacked_regions, name='Stacked Regions')

    # Auto-center and zoom
    viewer.camera.center = (stacked_regions.shape[2] // 2, stacked_regions.shape[1] // 2)
    viewer.camera.zoom = 3  # Adjust zoom level as needed

    

@magic_factory(
    image_layer={'label': 'Select Image Layer'},
    n_select={'label': 'Number of Images to Select', 'min': 1, 'max': 100, 'step': 1}
)
def select_random_images(image_layer: 'napari.layers.Image', n_select: int, viewer: 'napari.Viewer') -> None:
    global stacked_regions

    if stacked_regions is not None:
        # Randomly select n_select images from the stack
        selected_indices = np.random.choice(len(stacked_regions), size=min(n_select, len(stacked_regions)), replace=False)
        selected_images = stacked_regions[selected_indices]

        # Create a grid of selected images
        grid_size = int(np.ceil(np.sqrt(len(selected_images))))
        grid_img = np.zeros((grid_size * 50, grid_size * 50))

        for i, img in enumerate(selected_images):
            row = i // grid_size
            col = i % grid_size
            grid_img[row * 50:(row + 1) * 50, col * 50:(col + 1) * 50] = img

        # Add the grid image as a new layer
        viewer.add_image(grid_img, name='Selected Squares Grid')
        viewer.add_image(selected_images, name='Randomly Selected Squares')
    else:
        print("No regions available. Please run the blob center detection first.")

@magic_factory(image_stack={'label': 'Select Image Stack'})
def process_image(image_stack: 'napari.layers.Image', viewer: 'napari.Viewer') -> None:
    img_stack_data = image_stack.data

    radii = []
    img_stack_mean = []

    # Compute radius for each image in the stack
    for img_data in img_stack_data:
        # Normalize the image
        img_normalized = exposure.rescale_intensity(img_data, in_range='image', out_range=(0, 1))
        img_stack_mean.append(img_normalized)

        # Apply Sobel filter
        edges = filters.sobel(img_normalized)

        # Label connected regions
        labeled_edges = measure.label(edges > 0.1)
        regions = measure.regionprops(labeled_edges)

        if regions:
            # Find the largest region (assuming the largest one is the circle)
            largest_region = max(regions, key=lambda r: r.area)
            centroid = (largest_region.centroid[0], largest_region.centroid[1])
            
            # Create a radial profile from the Sobel-filtered image
            rr, cc = disk(centroid, largest_region.equivalent_diameter + 10, shape=edges.shape)
            profile = profile_line(edges, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')
            
            # Find peaks in the radial profile
            peaks, _ = find_peaks(profile, distance=10)
            
            if len(peaks) >= 2:
                # Calculate radius as half the distance between the two largest peaks
                peak_distances = np.diff(peaks)
                radius = peak_distances[np.argmax(peak_distances)] / 2
                radii.append(radius)
                
    # Compute the mean and standard deviation of the radii
    mean_radius = np.mean(radii) if radii else 0
    std_radius = np.std(radii) if radii else 0
    print(f"Mean Radius of the detected circles: {mean_radius:.2f} pixels")
    print(f"Standard Deviation of the Radii: {std_radius:.2f} pixels")

    # Compute mean image from the stack
    img_mean = np.mean(img_stack_mean, axis=0)
    img_mean_normalized = exposure.rescale_intensity(img_mean, in_range='image', out_range=(0, 1))
    
    # Add the normalized mean image as a new layer
    viewer.add_image(img_mean_normalized, name='Mean Image')

    # Apply Sobel filter to the mean image
    edges = filters.sobel(img_mean_normalized)

    # Label connected regions
    labeled_edges = measure.label(edges > 0.1)
    regions = measure.regionprops(labeled_edges)

    if regions:
        # Find the largest region (assuming the largest one is the circle)
        largest_region = max(regions, key=lambda r: r.area)
        centroid = (largest_region.centroid[0], largest_region.centroid[1])
        radius = int(largest_region.equivalent_diameter / 2)
        radplusten = radius + 10

        # Create a new image layer for the Sobel-filtered result
        viewer.add_image(edges, name='Sobel Filtered Image')

        # Create an overlay for the largest region
        regions_overlay = np.zeros_like(edges, dtype=np.int32)  # Ensure integer data type
        minr, minc, maxr, maxc = largest_region.bbox
        regions_overlay[minr:maxr, minc:maxc] = 1
        viewer.add_labels(regions_overlay, name='Detected Circle')

        # Calculate and display radial profile from the mean normalized image
        rr, cc = disk(centroid, radplusten, shape=img_mean_normalized.shape)
        profile = profile_line(img_mean_normalized, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')

        # Create the standard deviation profile
        std_profile = np.zeros_like(profile)
        for i in range(len(profile)):
            std_profile[i] = np.std([profile_line(img, (centroid[0], centroid[1]), (cc[-1], rr[-1]), mode='nearest')[i] for img in img_stack_mean])

        # Convert pixel indices to nanometers
        x_values_nm = np.arange(len(profile)) * 2  # 2 nanometers per pixel

        fig, ax = plt.subplots()
        ax.plot(x_values_nm, profile, label='Mean Intensity')
        ax.fill_between(x_values_nm, profile - std_profile, profile + std_profile, color='gray', alpha=0.5, label='Std Dev')
        ax.set_title(f'Radial Profile - Mean Radius: {mean_radius*2:.2f} nm, Std Dev: {std_radius*2:.2f} nm')
        ax.set_xlabel('Radius (nm)')
        ax.set_ylabel('Intensity')
        ax.legend()

        # Add plot to Napari viewer
        canvas = FigureCanvas(fig)
        viewer.window.add_dock_widget(canvas, name='Radial Profile')
    else:
        print("No regions found.")

' \n@magic_factory(image_stack={\'label\': \'Select Image Stack\'})\ndef process_image(image_stack: \'napari.layers.Image\', viewer: \'napari.Viewer\') -> None:\n    img_stack_data = image_stack.data\n\n    radii = []\n    img_stack_mean = []\n\n    # Compute radius for each image in the stack\n    for img_data in img_stack_data:\n        # Normalize the image\n        img_normalized = exposure.rescale_intensity(img_data, in_range=\'image\', out_range=(0, 1))\n        img_stack_mean.append(img_normalized)\n\n        # Apply Sobel filter\n        edges = filters.sobel(img_normalized)\n\n        # Label connected regions\n        labeled_edges = measure.label(edges > 0.1)\n        regions = measure.regionprops(labeled_edges)\n\n        if regions:\n            # Find the largest region (assuming the largest one is the circle)\n            largest_region = max(regions, key=lambda r: r.area)\n            centroid = (largest_region.centroid[0], largest_region.centroid[1])\n            \n   

### Initialize Napari Window and Dock Widgets

In [ ]:
# Initialize the viewer and widgets
viewer = napari.Viewer()

# Add the test_blob_centers widget
my_widget1 = test_blob_centers()
dw1 = viewer.window.add_dock_widget(my_widget1, name="Extract Filter and Classify")

# Add the select_random_images widget
my_widget2 = select_random_images()
dw2 = viewer.window.add_dock_widget(my_widget2, name="Select Random Images")

my_widget3 = process_image()
dw3 = viewer.window.add_dock_widget(my_widget3, name="Process Images")

@viewer.bind_key('d')
def delete_current_image(viewer):
    global stacked_regions
    if stacked_regions is not None:
        z_index = int(viewer.dims.point[0])

        # Remove the image at the current z-index
        stacked_regions = np.delete(stacked_regions, z_index, axis=0)

        # Update the viewer with the modified stack
        viewer.layers['Stacked Regions'].data = stacked_regions


## Testing Area

In [5]:
print(viewer.layers)

[<Image layer 'Randomly Selected Squares' at 0x3274cc1d0>]


In [6]:
img = viewer.layers[0]
imgdata = img.data
#mask = viewer.layers[1]
#maskdata = mask.data
print(np.shape(imgdata))
#print(np.shape(maskdata))
#deneme= np.array(data)
#viewer.add_image(deneme)

(50, 50, 50)


In [7]:
viewer.dims.point

(0.0, 49.0, 49.0)